# PKHND solution
#### Author : Hard_drive_corrupted(Pulkit Gera and Anchit Gupta)

## To open this notebook on Google Computing platform Colab, click below!


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/aicrowd-bot/005cd2a9231779a260fc4bd96159dff6)


## Solution
We simply check the different hands and programatically predict the label.
+ 0: Nothing in hand; not a recognized poker hand 
+ 1: One pair; one pair of equal ranks within five cards
+ 2: Two pairs; two pairs of equal ranks within five cards
+ 3: Three of a kind; three equal ranks within five cards
+ 4: Straight; five cards, sequentially ranked with no gaps
+ 5: Flush; five cards with the same suit
+ 6: Full house; pair + different rank three of a kind
+ 7: Four of a kind; four equal ranks within five cards
+ 8: Straight flush; straight + flush
+ 9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
We check for all and accordingly assign

## Download data
The first step is to download out train test data. We will be training a classifier on the train data and make predictions on test data. We submit our predictions


In [0]:
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/test.csv
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/train.zip
!unzip train.zip


## Import packages

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from tqdm import tqdm

## Load Data
We use pandas library to load our data. Pandas loads them into dataframes which helps us analyze our data easily. Learn more about it [here](https://www.tutorialspoint.com/python_data_science/python_pandas.htm)

In [0]:
train_data_path = "train.csv" #path where data is stored

In [0]:
train_data = pd.read_csv(train_data_path) #load data in dataframe using pandas

## Visualize data

In [0]:
train_data.head()

We can see there are 11 column where first 10 column contains the cards information and the last one describing the hand it makes. 1st and 2nd column contains suit and rank of first card respectively, 3rd and 4th column suit and rank of 2nd card and so on.

In [0]:
def hand_check(hand_data):
    val = 0
    suits = []
    ranks = []
    
    for i in range(0, 10, 2):
        suits.append(hand_data[i])
        ranks.append(hand_data[i+1])
        
    suits, = np.asarray(suits)
    ranks = np.asarray(ranks)
    
    uniq_r, counts_r = np.unique(ranks, return_counts=True)
    uniq_s, counts_s = np.unique(suits, return_counts=True)
    
    #checking for one pair, two pairs, three of a kind and four of a kind
    cnt = 0
    for i in range(len(counts_r)):
        if counts_r[i] > 1:
            cnt += 1
            if counts_r[i] == 3:
                val = max(val, 3)
            if counts_r[i] == 4:
                val = max(val, 7)
    if cnt == 1:
        val = max(val, 1)
    elif cnt == 2:
        val = max(val, 2)
        
    #checking for flush
    if len(uniq_s) == 1:
        val = max(val, 5)
    
    #checking for straight, straight flush and royal flush
    check = 1
    check2 = 0
    sorted_uniq_r = sorted(uniq_r)
    if len(uniq_r) == 5:
        for i in range(1, 5):
            if sorted_uniq_r[i] != sorted_uniq_r[i-1]+1:
                check = 0
        if sorted_uniq_r[0] == 1:
            if sorted_uniq_r[1] == 10 and sorted_uniq_r[2] == 11 and sorted_uniq_r[3] == 12 and sorted_uniq_r[4] == 13:
                check2 = 1
            
        if check == 1 or check2 == 1:
            val = max(val, 4)
            if len(uniq_s) == 1:
                val = max(val, 8)
                if check2 == 1:
                    val = max(val, 9)
                    
    #checking for full house
    cnt2 = 0
    cnt3 = 0
    for i in range(len(counts_r)):
        if counts_r[i] == 2:
            cnt2 = 1
        if counts_r[i] == 3:
            cnt3 = 1
    if cnt3 == cnt2 and cnt2 == 1:
        val = max(val, 6)
        
    return val


In [0]:
def run_model(data_frame, mode='train'):
    rows = data_frame.shape[0]
    
    res = []
    
    for i in tqdm(range(rows)):
        res.append(hand_check(np.array(data_frame.iloc[i, :])[0:10]))
    
    if mode == 'train':
        acc = 0
        for i in range(rows):
            if res[i] == np.array(data_frame.iloc[i, :])[10]:
                acc += 1
            
        return res, acc/rows
    elif mode == 'test':
        return res


In [19]:
res, acc = run_model(train_data)

100%|██████████| 500/500 [00:00<00:00, 3178.81it/s]


In [22]:
print("Accuracy : ",acc)

Accuracy :  1.0


## Load Test Set
Load the test data now

In [23]:
final_test_path = "test.csv"
final_test = pd.read_csv(final_test_path)
submission = run_model(final_test,'test')

100%|██████████| 25010/25010 [00:06<00:00, 3576.33it/s]


## Save the prediction to csv

In [0]:
submission = pd.DataFrame(submission)
submission.to_csv('submission.csv',header=['label'],index=False)

Note: Do take a look at the submission format.The submission file should contain a header.For eg here it is "label". 

## To download the generated csv in collab run the below command

In [0]:
from google.colab import files
files.download('submission.csv') 

### Go to [platform](https://www.aicrowd.com/challenges/aicrowd-blitz-may-2020/problems/pkhnd). Participate in the challenge and submit the submission.csv generated.